In [1]:
import matplotlib.pyplot as plt
import networkx as nx
import random
import pygraphviz as pgv
from subprocess import run
from networkx.algorithms.approximation.vertex_cover import min_weighted_vertex_cover

green = "green" #"darkturquoise"

In [2]:
def kernelize(G):
    nodes = list()
    for n in G.nodes:
        if nx.degree(G,n):
            nodes.append(n)
    return G.subgraph(nodes)
def color(A, nodes, color):
    for n in nodes:
        n = A.get_node(n)
        n.attr["fillcolor"] = color
    A.node_attr['style']='filled'

In [3]:
def bounded_search(G, k, solution=[]):
    if k == 0:
        return []
    v1, v2 = next(G.edges().__iter__())
    for v in (v1, v2):
        g = G.copy()
        g.remove_node(v)
        if len(g.edges()) == 0:
            return solution + [v]
    g1 = G.copy()
    g2 = G.copy()
    g1.remove_node(v1)
    g2.remove_node(v2)
    
    solution1 = bounded_search(g1, k-1, solution + [v1])
    if len(solution1):
        return solution1
    
    return bounded_search(g2, k-1, solution + [v2])

In [4]:
def k_vertex_cover(G, k):
    accepted = list()
    rejected = list()
    unknown = list()
    orig_nodes = [n for n in G.nodes()]
    success = True
    for v in orig_nodes:
            if G.degree(v) == 0:
                    accepted += [v]
            elif G.degree(v) == 1:
                    if len([e for e in G.subgraph(accepted + [v]).edges() if v in e]):
                            rejected += [v]
                            k -= 1
                    else: accepted += [v]
            elif G.degree(v) > k:
                    rejected += [v]
                    k -= 1
            else:
                    unknown += [v]
            if k < 0:
                    success = False
                    break
            # G = G.subgraph([n for n in G.nodes if n is not v])
    if len(G.subgraph(unknown + accepted).edges()) > k*k:
            success = False
    return success, accepted, rejected, unknown

In [5]:
# troublemaker
kernalize = False
G = nx.Graph()
N = 200
G.add_nodes_from(range(N))
troublemaker = "42"
G.add_node(troublemaker)
for x in range(30):
    x = troublemaker#  random.randint(1,N)
    y = random.randint(1,N)
    G.add_edge(x, y)
if kernalize:
    A = nx.nx_agraph.to_agraph(kernalize(G))
else:
        A = nx.nx_agraph.to_agraph(G)   
A.node_attr['style']='filled'
n = A.get_node(troublemaker)
n.attr["fillcolor"]="orange"
A.layout()
if kernalize:
    A.draw("vertex_troublemaker_kernelized.pdf")
else:
    A.draw("vertex_troublemaker.pdf")

In [6]:
# wild west
G = nx.Graph()
N = 20
G.add_nodes_from(range(N))
troublemaker = "42"
G.add_node(troublemaker)
for x in range(N):
    for y in range(x,N):
        G.add_edge(x, y)
A = nx.nx_agraph.to_agraph(G)   
A.node_attr['style']='filled'
A.node_attr['fillcolor']="white"
n = A.get_node(troublemaker)
n.attr["fillcolor"]="orange"
A.layout()
A.draw("vertex_wild_west.pdf")

In [7]:
# random

for kernel in (True, False):
    G = nx.Graph()
    N = 1000
    p = 0.01
    G.add_nodes_from(range(N))
    random.seed(44) # dont touch
    
    # normals
    for x in range(int(N*p*0.1)):
        x = random.randint(1,N)
        y = random.randint(1,N)
        G.add_edge(x, y)
    # stressful
    for x in range(int(N*p*0.5)):
        x = random.randint(1,N//100)
        y = random.randint(1,N//100)
        G.add_edge(x, y)
    # terror
    for x in range(int(N*p*0.4)):
        x = random.randint(1,N//500)
        y = random.randint(1,N//100)
        G.add_edge(x, y)
        
    # lord forgive me
    G.remove_node(4)
    
    if kernel:
        A = nx.nx_agraph.to_agraph(kernelize(G))
    else:
        A = nx.nx_agraph.to_agraph(G)
    #A.node_attr['style']='filled'
    #n = A.get_node(troublemaker)
    #n.attr["fillcolor"]="orange"
    dom = nx.algorithms.dominating.dominating_set(kernelize(G))
    cover = [n for n in kernelize(G).nodes if n not in dom]
    for n in A.nodes():
        n = A.get_node(n)
        n.attr["fillcolor"]="white"
    A.layout()
    if kernel:
        color(A,cover, "orange")
        A.draw("vertex_random_kernel.pdf")
        color(A, A.nodes(), "white")
        A.draw("vertex_random_kernel_nocolor.pdf")
        
    else:
        A.draw("vertex_random_nocolor.pdf")
        color(A,cover, "orange")
        A.draw("vertex_random.pdf")
        color(A, A.nodes(), "white")
        color(A,[n for n in A.nodes() if not A.degree(n)], green)
        A.draw("vertex_random_green.pdf")
run("./build")

CompletedProcess(args='./build', returncode=0)

In [8]:
# truly random

G = nx.Graph()
N = 100
p = 0.3
k = 8

use_k = True

G.add_nodes_from(range(N))
random.seed(42) 

# normals
for x in range(int(N*p)):
    x = random.randint(1,N)
    y = random.randint(1,N)
    G.add_edge(x, y)
    
success, accepted, cover, unknown = k_vertex_cover(kernelize(G), k)
dom = nx.algorithms.dominating.dominating_set(kernelize(G), 28)
cover2 = [n for n in kernelize(G).nodes if n not in dom]

print("Cover size: %d + %d unknowns" % (len(cover), len(unknown)))
print("Cover2 size: %d" % (len(cover2)))
print("Success?: %r" % success)
if not success:
    print("try again with k >= %d" % ((len(unknown)-k)**2 + k))
A = nx.nx_agraph.to_agraph(kernelize(G))
A.layout()
color(A, A.nodes(), "white")
color(A, cover2, "orange")
A.draw("random_kernel_nocolor.pdf")
color(A, A.nodes(), "white")
color(A, cover, "orange")
# color(A, unknown, "yellow")
color(A, accepted, green)
A.draw("random_kernel.pdf")
run("./build")

Cover size: 9 + 13 unknowns
Cover2 size: 18
Success?: False
try again with k >= 33


CompletedProcess(args='./build', returncode=0)

In [9]:
G = nx.Graph()
G.add_nodes_from(range(100))
G.add_edges_from((
    (1,2),
    (2,3),
    (2,4),
    (4,5),
    (7,8)
))
bounded_search(G, 4)

[1, 2, 4, 7]

In [10]:
next(G.edges().__iter__())

(1, 2)

In [11]:
bounded_search(G, 5)

[1, 2, 4, 7]

In [12]:
# bounded search tree
k = 20
G = nx.Graph()
G.add_nodes_from(range(400))
for i in range(200, 300):
    G.add_edge(42,i)
random.seed(42)
for i in range(20):
    G.add_edge(random.randint(0,100), random.randint(0,100))



In [13]:
success, accepted, cover, unknown = k_vertex_cover(kernelize(G), k)
print("Cover size: %d + %d unknowns" % (len(cover), len(unknown)))
print("Success?: %r" % success)
if not success:
    print("try again with k >= %d" % ((len(unknown)-k)**2 + k))



Cover size: 6 + 9 unknowns
Success?: True


In [14]:
tree = bounded_search(G, 20)
print("Tree: %r (%d)" % (tree, len(tree)))

Tree: [0, 3, 11, 13, 14, 17, 20, 25, 28, 29, 31, 42, 54, 57, 69, 83, 86] (17)


In [15]:
cover2 = min_weighted_vertex_cover(G)
print("Cover2 size: %d" % (len(cover2)))

Cover2 size: 22


In [17]:
from scipy.stats import binom
from matplotlib import pyplot as plt